In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import sqlite3
import pandas as pd
import xlsxwriter
from pandas import read_excel
import time
from dateutil.parser import parse
from functools import reduce
import datetime
from datetime import datetime,date,timedelta
import warnings
import calendar
c = calendar.Calendar(firstweekday=calendar.SUNDAY)
warnings.filterwarnings('ignore')
from WindPy import w
if w.isconnected()!=1:
    w.start()

ModuleNotFoundError: No module named 'pymssql'

In [1]:
import pandas as pd
all_fund_nav=pd.read_csv('all_fund_nav.csv')

In [2]:
all_fund_nav.head()

,Unnamed: 0,机构ID,机构简称,基金ID,基金简称,统计日期,复权累计净值,运行状态,基金策略
0,0,9151,梵基投资,6259,梵基一号基金(有限合伙),2011-04-01,1.000000,存续中,主观宏观
1,1,9151,梵基投资,6259,梵基一号基金(有限合伙),2011-04-22,0.933131,存续中,主观宏观
2,2,9151,梵基投资,6259,梵基一号基金(有限合伙),2011-04-29,1.022561,存续中,主观宏观
3,3,9151,梵基投资,6259,梵基一号基金(有限合伙),2011-05-06,1.095075,存续中,主观宏观
4,4,9151,梵基投资,6259,梵基一号基金(有限合伙),2011-05-13,1.026545,存续中,主观宏观


In [3]:
import datetime
from datetime import datetime,date,timedelta
all_fund_nav['统计日期']=[datetime.strptime(i,'%Y-%m-%d').date() for i in all_fund_nav['统计日期']]

In [4]:
all_fund_nav['统计日期'].head()

0    2011-04-01
1    2011-04-22
2    2011-04-29
3    2011-05-06
4    2011-05-13
Name: 统计日期, dtype: object

In [5]:
df_outside=pd.read_excel("C:/Users/Yin Hui/Desktop/导入净值new.xlsx", None)
df_outside=df_outside['Sheet1']
df_outside['统计日期']=[i.date() for i in df_outside['统计日期']]

In [6]:
all_fund_nav=all_fund_nav.append(df_outside)

In [8]:
all_fund_nav.to_csv('all_fund_nav.csv')

In [16]:
组合成分= pd.read_excel(r"C:\Users\Yin Hui\Desktop\自营组合展望.xlsx")

In [17]:
组合成分

,组合投资,Unnamed: 1,参考产品,代码,规模,预期收益,预期收益(费后）,预期波动
0,泰铼开源尊享1号,泰铼,泰铼金泰2号,504562,30000000,NaN,NaN,NaN
1,泰铼开源尊享2号,泰铼,泰铼信泰3号A,439678,33000000,NaN,NaN,NaN
2,橡木启明,橡木,橡木启明,500667,1000000,NaN,NaN,NaN
3,致远22号,致诚卓远,致远22号,291379,5000000,NaN,NaN,NaN
4,时代复兴开源微观八号,时代复兴,时代复兴微观一号,316470,1000000,NaN,NaN,NaN
5,富善投资开源歆享1号,富善,致远CTA精选5号,691375,16000000,NaN,NaN,NaN
6,珺容开源量化臻选1号,珺容,珺容翔宇CTA2号,442675,10000000,NaN,NaN,NaN
7,白鹭鼓浪屿量化多策略十号,白鹭,白鹭鼓浪屿量化多策略一号,435913,10000000,NaN,NaN,NaN
8,安诚数盈开源安进1号,安诚数盈,安诚数盈锐利1号,安诚数盈锐利1号,10000000,0.10,0.08,0.10
9,思想三十九号,思勰,思勰投资子张十号,472397,5000000,NaN,NaN,NaN


In [11]:
def get_trade_date(start_date,end_date,period='W'):
    data=w.tdays(start_date,end_date,period=period)
    trade_dates=data.Data[0]
    trade_dates=[i.date() for i in trade_dates]
    return trade_dates

def get_all_date(start_date,end_date):
    data=w.tdays(start_date, end_date, "Days=Alldays")
    all_dates=data.Data[0]
    all_dates=[i.date() for i in all_dates]
    return all_dates

In [12]:
all_week_date=get_trade_date('2005-05-04','',period='W')
all_date=get_all_date('2005-05-04','')

In [18]:
all_funds=[]
for i in range(len(组合成分)):
    #print(i)
    mask=all_fund_nav['基金ID']==组合成分['代码'][i]
    fund_info=all_fund_nav[mask][['基金简称','统计日期','复权累计净值']].dropna()
    #print(fund_info)
    fund_data_=pd.DataFrame(index=all_date,columns=[组合成分['参考产品'][i]])
        #把净值填充进有所有日期的表中
    fund_data_[组合成分['参考产品'][i]].loc[fund_info['统计日期']]=fund_info['复权累计净值'].tolist()
    fund_data_=fund_data_.fillna(method='ffill')
        #按照周度筛选出数据
    fund_data_=fund_data_.loc[all_week_date]
    
    #fund_data_=pd.DataFrame(index=all_week_date,columns=[组合成分['参考产品'][i]])
    #for j in all_week_date:
        #if j in fund_info['统计日期'].tolist():
            #fund_data_.loc[j][组合成分['参考产品'][i]]=fund_info[fund_info.统计日期==j].iloc[0][2]
    fund_data_=fund_data_.reset_index()
    #fund_data_['日期']=[datetime.strptime(i,'%Y-%m-%d') for i in fund_data_.index]
    #fund_data_=fund_data_.sort_values(by=['日期'],ascending=False)
    #print(fund_data_)
    #fund_info=fund_data_
    #fund_data_after_fee_=高水位法(fund_data_.iloc[:,1],费率)
    
    all_funds.append(fund_data_)

In [19]:
all_funds_merge=reduce(lambda left,right: pd.merge(left,right,on='index',how='outer'),all_funds)
all_funds_merge['index']=[datetime.strptime(str(i),'%Y-%m-%d') for i in all_funds_merge['index']]
all_funds_merge=all_funds_merge.sort_values(by=['index'],axis=0,ascending=True, inplace=False)
all_funds_merge=all_funds_merge.set_index('index',drop=True)

In [20]:
all_funds_merge

,泰铼金泰2号,泰铼信泰3号A,橡木启明,致远22号,时代复兴微观一号,致远CTA精选5号,珺容翔宇CTA2号,白鹭鼓浪屿量化多策略一号,安诚数盈锐利1号,思勰投资子张十号,时间序列量化对冲1号,旭诺CTA三号,衍合量化市场中性1号,无隅鲲鹏一号,图灵谷雨中性一号,殊馥馥源套利1号,聊塑投资期权1号
index,,,,,,,,,,,,,,,,,
2005-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-29,1.3432,1.392277,1.064,2.035318,1.451113,1.029,1.738875,1.503316,1.698233,1.2108,1.172314,2.1457,1.127946,2.154513,1.088,1.342457,1.5469
2022-08-05,1.3198,1.387182,1.056,2.035318,1.457236,1.027,1.729576,1.483177,1.640632,1.2080,1.172314,2.0661,1.117118,2.133539,1.088,1.342457,1.5480
2022-08-12,1.3709,1.418000,1.069,2.037079,1.456011,1.038,1.742067,1.493839,1.671569,1.2154,1.170224,2.0743,1.119056,2.193925,1.084,1.342457,1.5473


In [86]:
all_funds_merge.to_excel('portfolio_nav.xlsx')

In [72]:
all_fund_return=np.array((all_funds_merge/all_funds_merge.shift()).sub(1).mean()*52*0.8)

In [85]:
all_fund_weekly_return=(all_funds_merge/all_funds_merge.shift()).sub(1)

In [73]:
port_weight=np.array(组合成分['规模']/组合成分['规模'].sum())

In [30]:
port_return=all_fund_return*port_weight.T

In [64]:
import numpy as np

In [74]:
port_return=np.sum(all_fund_return*port_weight)

In [75]:
port_return

0.09691383365205

In [90]:
port_std=np.dot(port_weight.T,np.dot(all_fund_weekly_return.cov()*52,port_weight))**0.5

In [91]:
port_std

0.0447061825800149

In [92]:
port_return/port_std

2.1677948788983294